## LDA 主题模型简介

### Reference

- 通俗理解LDA主题模型 https://blog.csdn.net/v_july_v/article/details/41209515
- David M. Blei所写的《Introduction to Probabilistic Topic Models》的译文 http://www.cnblogs.com/siegfang/archive/2013/01/30/2882391.html
- gensim https://blog.csdn.net/sinat_33761963/article/details/53945581
- 搜狗实验室语料 http://www.sogou.com/labs/resource/list_yuliao.php
- http://www.cnblogs.com/siegfang/archive/2013/01/30/2882391.html
- gensim training tips: https://miningthedetails.com/blog/python/lda/GensimLDA/

### 前提知识

- 多项分布
- 狄利克雷分布 (Dirichlet Distribution)
- 先验共轭分布
- 贝叶斯估计
- Gibbs采样

### 介绍

LDA模型现在已经成为了主题建模中的一个标准, 是实践中最成功的主题模型之一. 每一篇文档代表了一些主题所构成的一个概率分布, 而每一个主题又代表了很多单词所构成的一个概率分布. 

简单的描述, LDA可以被认为是如下的一个聚类过程:
- 各个主题 Topic 对应各类的'质心', 每一篇文档被视为数据集中的一个样本
- 主题和文档都被认为存在于同一个向量空间中, 这个向量空间中, 每一个特征向量都是词频(词袋模型)
- 与传统聚类中采用距离或者密度公式来衡量不同, LDA 使用一个基于统计模型的方式(这个统计模型揭示了文档是怎么产生的)

生成文档可以看做如下过程:

- 随机选择一个主题集合
- 对于文档中的每一个词:
    - 1. 从主题集合中随机选择一个主题
    - 2. 从主题对应的词袋中根据分布选择一个词
    
而主题模型建模的中心问题就是利用看到的文档推断出隐藏的主题结构, 也就是生成文档的逆过程.

<img src="images/lda.png" width="800"/>

### LDA 和 概率模型

> LDA和其它主题模型都属于概率建模这一更大领域. 数据被看作是经过包括隐藏变量在内的生成过程得到的. 生成过程定义了观测随机变量和隐藏随机变量的联合概率分布. 通过使用联合分布来计算在给定观测变量下隐藏变量的条件分布（后验分布）来进行数据分析. 对于LDA来说，观测变量就是文档中的词; 隐藏变量就是主题结构; 生成过程如之前所述. 那么推测从文档中隐藏的主题结构的问题其实就是计算在给定文档下隐藏变量的条件分布（后验分布）.

<img src="images/lda2.png" width="600" />

更具体一点描述 LDA 生成文档的过程如下:
- 从参数为 $\alpha$ 的狄利克雷分布 取样生成文档 $d$ 的主题分布 $\theta_d$
- 从主题的多项分布 $\theta_d$ 中取样生成文档 $d$ 中, 第 $n$ 个词的主题 $Z_{d,n}$
- 从参数为 $\eta$ 的狄利克雷分布 取样生成主题 $k$ 中的词的分布 $\beta_k$
- 对于文档 $d$ 第 $n$ 个位置, 由 $Z_{d,n}$ 和 $\beta_k$, 即这个位置上的主题分布和某主题的词分布, 就可采样生成这个位置的词 $W_{d,n}$


### 如何估计 LDA 模型参数

LDA的原始论文中是用的变分-EM算法估计未知参数, 后来发现了一种更好的估计参数的方法, 就是 Gibbs Sampling. Gibbs抽样是马尔可夫链蒙特卡尔理论（MCMC）中用来获取一系列近似等于指定多维概率分布（比如2个或者多个随机变量的联合概率分布）观察样本的算法。

### 缺点和不足
- 对短文本的提取效果有限
- 因为使用词袋模型和 Unigram 模型, 单独的考虑每个词, 而没有相互关系; 可以用 N-gram 模型来改进
- 主题重叠, 可以对相似主题进行合并

### 主题模型应用

文档的主题分布可看做是包含语义信息的一个降维过程, 得到文档的低维语义表示

#### 语义表达应用

- 作为表示语义的 Embedding 信息, 可作为特征用于其它机器学习任务. 比如新闻质量评估(分类)
- 作为语义空间的向量表示, 可用于聚类任务. 比如新闻聚类
- 通过一篇文档的主题分布计算信息熵, 可以表示内容的丰富程度, 或者专注度.
- 作为低纬度的隐式表达, 来描述用户的画像; 并通过计算和新内容主题表示的 Hellinger Distance 来作为推荐的依据.

#### 语义匹配应用

- 计算文档的相似度
- 计算文档和词的相似度, 对无语义的关键词抽取方法(TF-IDF)进行校准, 添加部分语义信息.

> 注: Hellinger Distance 通常用于计算分布的相似度

### 例子

**以下是两个例子, 对文档进行主题的提取, 可以直观的看到一个文档的语义信息由主题分布表示, 而一个主题又由词分布表示.**

> 目前，拿到了一份关于全球新能源汽车市场情况的报告。整理>了下数据，为应对中国新能源汽车中长期市场情况提供一点参考。先上结论1.日系车企尤其是丰田、本田在HEV（油电混合动力）方面有明显优势。欧美系车企除福特侧重HEV方面外，奔驰、大众、通用、>宝马、沃尔沃等都主要侧重于PHEV（插电式混合动力）/BEV（纯电动）。而各国家的扶持政策大都集中在PHEV/BEV领域。2.由于中国的政策主要侧重于PHEV/BEV产品，长远来看，大众、奔驰、宝马、通用>等产品及规划将更加适合中国市场，而丰田、本田等日韩系车企及福特公司在中国新能源市场将有掉队的可能。3.外资品牌在新能源汽车上近年来也投入了大量的精力，随着中国政府对自主品牌扶持力度>逐渐减弱，未来将面临外资品牌激烈的竞争。详细数据的分割线01 日本在HEV车型方面起步较早，并保持较高水平。其他国家相对日本来讲竞争力差距较大。而在PHEV/BEV方面，以中国为主的其他国家则>增长非常迅速，而日本近年来几乎原地不动。02 在汽车主要生产消费国中，日本在HEV方面同样占据领先地位，中美两国近两年来也同样占据一定的份额。而BEV/PHEV方面，由于政府巨大投入和政策支持>，中国市场遥遥领先。03 从各主要制造商方面来看，同样显示出类似的趋势。丰田作为最大的日系主机厂，占据HEV最大的份额，另一个日系品牌本田位居次席。而在BEV/PHEV方面，中国品牌增长极快，>产销量已远远领先于其他地区品牌。04 各主机厂规划方面，日系丰田、本田仍侧重于HEV方面，而日产、三菱及雷诺的联盟在BEV方面也在加大投入。其他欧美系（除福特）都将PHEV/BEV作为主要发展方向。05 各主要汽车生产国政府也将扶持补贴倾向于PHEV/BEV。对未来全球新能源市场基本形成共识。由于拿到的资料数据有限，分析不够全面。仅供各位同仁参考。

    # 主题分布
    [(956, 0.12760416666666669), (121, 0.059548611111111115), (960, 0.05763888888888889), (999, 0.05277777777777778), (199, 0.03802083333333334),...]
    
    # 主题的词向量表示
    Document Topic Distribution:
    =====> topic_id: 956, 0.127604166667 <=====
    2529	东风	0.0672594758981
    984	车型	0.0624917562891
    2445	suv	0.0592373839181
    108	汽车	0.0331256190066
    144	品牌	0.0283300513848
    5702	广汽	0.0276480663692
    5739	吉利	0.0253923773347
    6232	本田	0.0243359006327
    4828	车展	0.0236027638279
    6143	日产	0.0232596157207
    =====> topic_id: 121, 0.0595486111111 <=====
    108	汽车	0.783632918346
    3517	零部件	0.0235878270689
    4793	整车	0.0193733731056
    4960	车企	0.0127796306628
    1621	厂商	0.00681153445541
    14221	网联	0.00550073597039
    4277	制造商	0.00525926056398
    14774	造车	0.00522415201549
    8633	保有量	0.00433179982367
    22115	汽车业	0.00386158329795
    =====> topic_id: 960, 0.0576388888889 <=====
    3	市场	0.611017933898
    582	份额	0.132218564129
    1764	占据	0.0411131370759
    349	竞争	0.0247516971705
    5060	抢占	0.0206105684054
    6121	市场占有率	0.0125043489577
    202	优势	0.0121157281782
    125	国内	0.0116893743413
    4576	占有	0.00766367307841
    8811	先机	0.00735111263496
    =====> topic_id: 999, 0.0527777777778 <=====
    467	动力	0.177664188489
    1448	发动机	0.11649542113
    3298	燃料	0.0706256390226
    1845	混合	0.0423665906394
    4905	电机	0.0415635319911
    69	系统	0.0198771927809
    7029	油耗	0.0181051615794
    6201	燃油	0.0161692047886
    1647	性能	0.0146876886138
    1124	驱动	0.0143158735643
    =====> topic_id: 199, 0.0380208333333 <=====
    4124	丰田	0.106365153422
    3567	长安	0.0838340170242
    5706	福特	0.0676677674252
    5836	改装	0.0365952106782
    8147	豪车	0.0297361823299
    9169	跑车	0.0291596838561
    14619	雷克萨斯	0.019205160876
    108	汽车	0.0183688529665
    15382	林肯	0.0176723740806
    15508	皮卡	0.0173675349595



> 猴子聊军事自从二战结束后，斯大林通过雅塔尔会议将南千岛群岛及其附属岛屿变为苏联领土的一部分，苏联解体后俄罗斯继承它的遗产。随后，日本和俄国就有领土争端，日>本历届首相一直想要夺回二战中丢出去的北方四岛(俄罗斯称南千岛群岛)，然而普京警告到，俄罗斯领土虽大但是没有一寸是多余的，并且它们是苏俄士兵用鲜血换来的，如果谁想夺取，俄罗斯不惧战争>。面对俄罗斯的强势态度，日本并没有被退让反而更加猖狂，尤其到安倍这届政府对夺取北方领土的野心更加强烈，这不在今年2月份，安倍竟然高喊假若和平方式解决不了北方领土问题，那么日本只能动用武力解决。事实确实如此，安倍的开始大肆增加军费防务支出，今年军费竟高达460亿美元。他还在长崎和广岛73年纪念大会上只字未提无核三原则，从此事可以看出日本对二战犯下的罪孽毫无忏悔之心。同时一些日本右翼分子更是肆无忌惮竟多次攻击驻日俄罗斯大使，并叫嚣3天真夺回北方四岛。安倍和特朗普对于日本的野心，美国竟推波助澜欲向其推送世界最强五代机F22来维护亚太地区的稳定。据>《日本经济新闻亚洲评论》网站23日报道，日本是美国亚太地区最忠诚的盟友，美国10年前一直反对向日本出手F-22“猛禽”，然而现在要对付俄罗斯和日本五代机项目利益的驱使，美国终于妥协了，美军>将向日本派出还未装备的升级版F-22战斗机，来让日本提前得到其性能。如此一来，日本掌握最强五代机F22只是时间问题，倘若日本空军装备此战机，届时亚太地区以及北方四岛上空的平衡将被打破。美国此举更加危险将更加危险，难道特朗普忘了日本偷袭珍珠港造成大量美军死亡的事情了吗？美国这样放纵日本让其羽翼渐丰，迟早有一天会搬起石头砸自己的脚，毕竟日本现在还记得广岛和长崎的仇恨>！
    
    # 主题分布
    [(213, 0.18333333333333332), (1408, 0.05161290322580645), (1112, 0.048745519713261645), (1342, 0.048566308243727596), (250, 0.03799283154121864),...]
    
    # 主题的词向量表示
    Document Topic Distribution:
    =====> topic_id: 213, 0.183333333333 <=====
    177	日本	0.208757775042
    3478	安倍	0.0629084474871
    3471	首相	0.0279136185116
    8730	慰安妇	0.0183889932259
    8690	安倍晋三	0.0179732756378
    102	报道	0.0156681249167
    8344	日方	0.0153130796253
    10967	自民党	0.0134056695155
    47	政府	0.0133783074994
    7343	防卫	0.0125554642607
    =====> topic_id: 1408, 0.0516129032258 <=====
    480	俄罗斯	0.401462481573
    3302	普京	0.0680638083637
    4328	乌克兰	0.0436836817094
    4463	北约	0.0424852001812
    5070	莫斯科	0.0368328293406
    538	总统	0.0225712224862
    7693	俄方	0.0206301566298
    3516	制裁	0.0169910623132
    8831	远东	0.0155585697222
    1906	西方	0.0106976870761
    =====> topic_id: 1112, 0.0487455197133 <=====
    1532	战争	0.0766286969316
    25	美国	0.0436801577245
    13	国家	0.028776197293
    1049	军事	0.0276830882876
    120	关系	0.0212538043717
    2118	外交	0.0194637901261
    253	政治	0.0193182331802
    2344	大国	0.0158926021336
    2712	对抗	0.0152950591199
    139	战略	0.0148233998292
    =====> topic_id: 1342, 0.0485663082437 <=====
    1049	军事	0.130996920369
    2930	美军	0.0907056455858
    25	美国	0.0841643293139
    4341	国防部	0.0447813476214
    1666	军队	0.0254147187735
    4762	军方	0.0234432409611
    7559	防务	0.018934788125
    2099	国防	0.0151188864648
    10866	国防部长	0.0149959367714
    432	部署	0.014496821688
    =====> topic_id: 250, 0.0379928315412 <=====
    5560	面子	0.0172778586718
    7068	委屈	0.01024137887
    1921	明白	0.00891696519311
    11494	嘲笑	0.00784064053355
    923	不好	0.00739055714481
    10031	得罪	0.00656555106369
    13682	嫉妒	0.00644664449328
    8927	在乎	0.00607583215149
    7514	可怜	0.00576961103312
    4652	想想	0.00570780897615